# Phase 2 - DATA CLEANING & PREPROCESSING

In [90]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

plt.style.use("ggplot")
pd.set_option("display.max_columns", 100)

In [91]:
df = pd.read_csv("/Users/hibrizi/Project/hotel_booking_user_segmentation/data/hotel_bookings.csv")
print("Before cleaning:", df.shape)
df.head()

Before cleaning: (119390, 32)


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [92]:
print(f"Duplicate value: {df.duplicated().sum()}")
before_dupes = df.shape[0]
df = df.drop_duplicates().reset_index(drop=True)
after_dupes = df.shape[0]
print(f"After removing duplicates: {df.shape} (removed {before_dupes - after_dupes} rows)")

Duplicate value: 31994
After removing duplicates: (87396, 32) (removed 31994 rows)


In [93]:
mask = (df["adults"] + df["children"].fillna(0) + df["babies"]) == 0
print(f"Rows with zero guests: {mask.sum()}")
df = df[~mask]

Rows with zero guests: 166


In [94]:
df["reservation_status_date"] = pd.to_datetime(df["reservation_status_date"], errors="coerce")

df["arrival_date"] = pd.to_datetime(
    df["arrival_date_year"].astype(str) + "-" +
    df["arrival_date_month"].astype(str) + "-" +
    df["arrival_date_day_of_month"].astype(str),
    errors="coerce"
)

cat_cols = ["hotel","meal","country","market_segment","distribution_channel",
            "reserved_room_type","assigned_room_type","deposit_type",
            "agent","customer_type","reservation_status"]
for col in cat_cols:
    if col in df.columns:
        df[col] = df[col].astype(str)

In [95]:
fill_map = {"country": "Unknown", "agent": "NoAgent"}
for col, val in fill_map.items():
    if col in df.columns:
        df[col] = (
            df[col]
            .fillna(val)
            .replace(["nan", "NaN", "None", "NULL"], val)
            .astype(str)
        )

if "company" in df.columns:
    df.drop(columns=["company"], inplace=True)
    
num_cols = df.select_dtypes(include=["int64", "float64"]).columns
for col in num_cols:
    if df[col].isna().sum() > 0:
        df[col] = df[col].fillna(df[col].median())

In [96]:
invalid_adr = (df["adr"] < 0).sum()
if invalid_adr > 0:
    print(f"Found {invalid_adr} negative ADR values → replaced with median.")
    df.loc[df["adr"] < 0, "adr"] = df["adr"].median()

invalid_lead = (df["lead_time"] < 0).sum()
if invalid_lead > 0:
    df.loc[df["lead_time"] < 0, "lead_time"] = 0

Found 1 negative ADR values → replaced with median.


In [97]:
adr_cap = df["adr"].quantile(0.99)
df.loc[df["adr"] > adr_cap, "adr"] = adr_cap

lt_cap = df["lead_time"].quantile(0.99)
df.loc[df["lead_time"] > lt_cap, "lead_time"] = lt_cap

In [98]:
df["total_nights"] = df["stays_in_weekend_nights"] + df["stays_in_week_nights"]
df["total_guests"] = df["adults"] + df["children"] + df["babies"]
df["total_revenue"] = df["adr"] * df["total_nights"]
df["used_promo"] = (df["market_segment"].str.contains("Online", case=False)).astype(int)

In [99]:
for col in ["market_segment", "distribution_channel", "customer_type", "deposit_type"]:
    if col in df.columns:
        le = LabelEncoder()
        df[col + "_enc"] = le.fit_transform(df[col])

In [100]:
print("\nAfter cleaning:", df.shape)
print("Remaining Missing Values:", df.isna().sum().sum())
print("\nPreview after cleaning:")
display(df.head())


After cleaning: (87230, 40)
Remaining Missing Values: 0

Preview after cleaning:


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,arrival_date,total_nights,total_guests,total_revenue,used_promo,market_segment_enc,distribution_channel_enc,customer_type_enc,deposit_type_enc
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NoAgent,0,Transient,0.0,0,0,Check-Out,2015-07-01,2015-07-01,0,2.0,0.0,0,3,1,2,0
1,Resort Hotel,0,347,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NoAgent,0,Transient,0.0,0,0,Check-Out,2015-07-01,2015-07-01,0,2.0,0.0,0,3,1,2,0
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NoAgent,0,Transient,75.0,0,0,Check-Out,2015-07-02,2015-07-01,1,1.0,75.0,0,3,1,2,0
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,0,Transient,75.0,0,0,Check-Out,2015-07-02,2015-07-01,1,1.0,75.0,0,2,0,2,0
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,0,Transient,98.0,0,1,Check-Out,2015-07-03,2015-07-01,2,2.0,196.0,1,6,3,2,0


In [101]:
df.to_csv("/Users/hibrizi/Project/hotel_booking_user_segmentation/data/hotel_bookings_cleaned.csv")

# Data Cleaning & Preprocessing

### Objective
Prepare the `hotel_bookings` dataset for feature engineering and modeling.  
Core steps include removing duplicates, fixing data types, handling missing values, treating outliers, and creating new meaningful features such as `total_nights` and `total_revenue`.

---

## 1. Remove Duplicates
**Why:**  
Duplicate records may overweight certain customer patterns and introduce bias into clustering or prediction models.

**Action:**  
Use `drop_duplicates()` to ensure each booking record is unique.

---

## 2. Handle Missing Values
Several columns contain missing values, with some having extremely high proportions.

### Strategy:
- **Drop columns with excessive missing values (>90%)**  
  - `company` — ~94% missing → remove from dataset.

- **Impute categorical missing values:**  
  - `country` → `"Unknown"`  
  - `agent` → `"Unknown"`

- **Impute numerical missing values:**  
  - Columns such as `children`, `babies`, and other count-based variables → use **median imputation** for stability.

**Why:**  
Imputation avoids unnecessary loss of data while ensuring cleaner input for clustering algorithms.

---

## 3. Fix Data Types & Combine Dates
### Convert separate date columns into a single datetime field:
Combine:
- `arrival_date_year`
- `arrival_date_month`
- `arrival_date_day_of_month`

into:

`arrival_date = pd.to_datetime(...)`

### Convert reservation status date:
- `reservation_status_date` → `datetime`

**Why:**  
Correct date formatting allows calculation of:
- recency,  
- seasonality patterns,  
- length between booking and check-in.

---

## 4. Outlier Treatment
Certain numerical features contain extreme values (e.g., ADR > 5000).

### Recommended approach:
- Apply winsorization (capping) at the **99th percentile** for:
  - `adr`
  - `lead_time`

**Why:**  
K-Means and other distance-based algorithms are highly sensitive to outliers and gaps in scale.

---

## 5. Feature Creation (Feature Engineering)
Enhance the dataset by adding features that better reflect customer behavior.

### New columns:
- **`total_nights`**  
  `stays_in_week_nights + stays_in_weekend_nights`

- **`total_guests`**  
  `adults + children + babies`

- **`total_revenue`**  
  `adr × total_nights`  
  Represents customer value contribution.

- **`used_promo`**  
  `1` if `market_segment` refers to an Online channel, else `0`  
  (Indicator for promo-driven behavior)

**Why:**  
These engineered features capture spending behavior, group size, length of stay, and promo sensitivity—important for segmentation and clustering.

---

## ✔ Summary of Preprocessing Goals
| Step | Purpose | Outcome |
|------|---------|---------|
| Remove duplicates | Avoid bias | Clean unique entries |
| Impute missing values | Preserve dataset size | More reliable analysis |
| Fix data types | Enable time-based calculations | Better temporal modeling |
| Treat outliers | Improve algorithm stability | Scaled, consistent input data |
| Create new features | Capture real behavior patterns | Stronger clustering performance |

---

Dataset is now ready for **feature scaling → clustering → interpretation**.
